## Authenticate to Google Earth Engine

In [1]:
import ee
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='ee-raman')

In [2]:
def get_feature_collection(asset_id):
    """Check if an asset exists, and load it as a FeatureCollection if it does.
    Otherwise, return an empty FeatureCollection.
    
    Args:
        asset_id (str): The Earth Engine asset ID.
        
    Returns:
        ee.FeatureCollection: The loaded FeatureCollection or an empty one.
    """
    try:
        # Get asset information to check existence
        ee.data.getAsset(asset_id)
        print(f"Asset '{asset_id}' exists. Loading FeatureCollection.")
        return ee.FeatureCollection(asset_id)
    except Exception as e:
        print(f"Asset '{asset_id}' does not exist. Returning empty FeatureCollection.")
        return ee.FeatureCollection([])

In [6]:
filename_prefix = "Area_bheramgarh"
suffix = filename_prefix.split("_")[-1]
mapping = {
    "farm": 1,
    "plantation": 2,
    "scrubland": 3,
    "rest": 0
}
reversed_mapping = {v: k for k, v in mapping.items()}
reversed_ee_mapping = ee.Dictionary(reversed_mapping)

def get_class_and_mapping(vectors, vector_name):
    vector_of_class = vectors.filter(ee.Filter.eq('class', vector_name))
    vector_of_class_with_map = vector_of_class.map(lambda f: f.set('class', mapping[vector_name]))
    return vector_of_class, vector_of_class_with_map

all = get_feature_collection("projects/ee-raman/assets/all_" + suffix)


ts_data = ee.Image("projects/ee-raman/assets/ts_data_" + suffix)


Asset 'projects/ee-raman/assets/all_bheramgarh' exists. Loading FeatureCollection.


In [4]:
import math
from itertools import product


# Function to convert latitude to pixel Y at a given zoom level
def lat_to_pixel_y(lat, zoom):
    sin_lat = math.sin(math.radians(lat))
    pixel_y = ((0.5 - math.log((1 + sin_lat) / (1 - sin_lat)) / (4 * math.pi)) * (2 ** (zoom + 8)))
    return pixel_y

# Function to convert longitude to pixel X at a given zoom level
def lon_to_pixel_x(lon, zoom):
    pixel_x = ((lon + 180) / 360) * (2 ** (zoom + 8))
    return pixel_x

# Function to convert pixel X to longitude
def pixel_x_to_lon(pixel_x, zoom):
    lon = (pixel_x / (2 ** (zoom + 8))) * 360 - 180
    return lon

# Function to convert pixel Y to latitude
def pixel_y_to_lat(pixel_y, zoom):
    n = math.pi - 2 * math.pi * pixel_y / (2 ** (zoom + 8))
    lat = math.degrees(math.atan(math.sinh(n)))
    return lat

def lat_lon_from_pixel(lat, lon, zoom, scale):
    """
    Given a starting latitude and longitude, calculate the latitude and longitude
    of the opposite corner of a 256x256 image at a given zoom level.
    """
    pixel_x = lon_to_pixel_x(lon, zoom)
    pixel_y = lat_to_pixel_y(lat, zoom)
    
    new_lon = pixel_x_to_lon(pixel_x + 256*scale, zoom)
    new_lat = pixel_y_to_lat(pixel_y + 256*scale, zoom)

    return new_lat, new_lon

"""

Helper function for dividing an roi into blocks

"""
def get_n_boxes(lat, lon, n, zoom, scale):
    diagonal_lat_lon = [(lat, lon),]
    for i in range(n):
        new_lat_lon = lat_lon_from_pixel(lat, lon, zoom, scale)
        diagonal_lat_lon.append(new_lat_lon)
        lat, lon = new_lat_lon
    lats = [i[0] for i in diagonal_lat_lon]
    longs = [i[1] for i in diagonal_lat_lon]
    return list(product(lats, longs))

def get_points(roi):
    zoom = 17
    scale = 16
    bounds = roi.bounds().coordinates().get(0).getInfo()
    lons = sorted([i[0] for i in bounds])
    lats = sorted([i[1] for i in bounds])
    starting_point = lats[-1], lons[0]
    min_, max_ = (
        [lon_to_pixel_x(lons[0], zoom), lat_to_pixel_y(lats[0], zoom) ],
        [lon_to_pixel_x(lons[-1], zoom), lat_to_pixel_y(lats[-1], zoom)]
        )
    iterations = math.ceil(max(abs(min_[0] -  max_[0]), abs(min_[1] - max_[1]))/256/16)
    points = get_n_boxes(starting_point[0], starting_point[1], iterations, zoom, scale)
    intersect_list = []
    print(len(points))
    for point in points:
        top_left = point
        bottom_right = lat_lon_from_pixel(top_left[0], top_left[1], zoom, scale)
        rectangle = ee.Geometry.Rectangle([(top_left[1], top_left[0]), (bottom_right[1], bottom_right[0])])
        print(top_left, bottom_right)
        intersects = roi.geometry().intersects(rectangle, ee.ErrorMargin(1)).getInfo()
        if intersects:
            intersect_list.append([top_left, bottom_right])
        print(intersects)
    return intersect_list

roi_boundary = ee.FeatureCollection("users/mtpictd/india_block_boundaries").filter(ee.Filter.eq("block", "Bheramgarh"))
filename_prefix = "Area_bheramgarh"

points = get_points(roi_boundary)

roi_boundary = ee.FeatureCollection([ee.Feature(ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])) for top_left, bottom_right in points])


256
(19.35210359962618, 80.60491122661094) (19.31063593149325, 80.64885653911097)
False
(19.35210359962618, 80.64885653911097) (19.31063593149325, 80.69280185161097)
False
(19.35210359962618, 80.69280185161097) (19.31063593149325, 80.73674716411097)
False
(19.35210359962618, 80.73674716411097) (19.31063593149325, 80.78069247661097)
True
(19.35210359962618, 80.78069247661097) (19.31063593149325, 80.82463778911097)
True
(19.35210359962618, 80.82463778911097) (19.31063593149325, 80.86858310161097)
True
(19.35210359962618, 80.86858310161097) (19.31063593149325, 80.91252841411097)
True
(19.35210359962618, 80.91252841411097) (19.31063593149325, 80.95647372661097)
True
(19.35210359962618, 80.95647372661097) (19.31063593149325, 81.00041903911097)
True
(19.35210359962618, 81.00041903911097) (19.31063593149325, 81.04436435161097)
False
(19.35210359962618, 81.04436435161097) (19.31063593149325, 81.08830966411097)
False
(19.35210359962618, 81.08830966411097) (19.31063593149325, 81.13225497661097)


In [11]:
result = ee.FeatureCollection([])

for ind, (top_left, bottom_right) in enumerate(points):
    roi_boundary = ee.FeatureCollection([ee.Feature(ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]]))])
    
    all_ = all.filter(ee.Filter.eq('block', ind))
    ts_data_ = ts_data.clip(roi_boundary)
    farm, farm_with_map = get_class_and_mapping(all_, "farm")
    scrubland, scrubland_with_map = get_class_and_mapping(all_, "scrubland")
    plantation, plantation_with_map = get_class_and_mapping(all_, "plantation")
    rest, rest_with_map = get_class_and_mapping(all_, "rest")
    
    training_features = farm_with_map.merge(scrubland_with_map).merge(plantation_with_map)
    unique_classes = training_features.aggregate_array('class').distinct().getInfo()
    if len(unique_classes)>1:
        training_data = ts_data_.sampleRegions(
            collection=training_features,
            properties=['class'],
            scale=10  # Adjust scale based on resolution
        )
        classifier = ee.Classifier.smileRandomForest(50).train(
            features=training_data,
            classProperty='class',
            inputProperties=ts_data_.bandNames()
        )

        classified = ts_data_.classify(classifier)

        def assign_mode_label(feature):
            class_values = classified.reduceRegion(
                reducer=ee.Reducer.mode(),
                geometry=feature.geometry(),
                scale=30,  # Adjust scale as per resolution
                bestEffort=True
            )
            return feature.set('class', class_values.get('classification'))

        # Apply function to test features
        rest_with_labels = rest_with_map.map(assign_mode_label).filter(ee.Filter.notNull(['class']))
        rest_with_labels = rest_with_labels.map(lambda x: x.set('class', reversed_ee_mapping.get(ee.Number(x.get("class")).int())))
        all_with_labels = rest_with_labels.merge(farm).merge(scrubland)
        result = result.merge(all_with_labels)
        print("Done ", ind)
    else:
        print("Single Class",ind)
        rest_with_labels = rest_with_map.map(lambda x: x.set("class", ee.Number(unique_classes[0]))).filter(ee.Filter.notNull(['class']))
        rest_with_labels = rest_with_labels.map(lambda x: x.set('class', reversed_ee_mapping.get(ee.Number(x.get("class")).int())))
        all_with_labels = rest_with_labels.merge(farm).merge(scrubland)
        result = result.merge(all_with_labels)

Done  0
Done  1
Done  2
Done  3
Done  4
Done  5
Done  6
Done  7
Done  8
Done  9
Done  10
Done  11
Done  12
Done  13
Done  14
Done  15
Done  16
Done  17
Done  18
Done  19
Done  20
Done  21
Done  22
Done  23
Done  24
Done  25
Done  26
Done  27
Done  28
Done  29
Done  30
Done  31
Done  32
Done  33
Done  34
Done  35
Done  36
Done  37
Single Class 38
Done  39
Done  40
Done  41
Done  42
Done  43
Done  44
Done  45
Done  46
Done  47
Done  48
Done  49
Done  50
Done  51
Done  52
Done  53
Done  54
Done  55
Done  56
Done  57
Done  58
Done  59
Done  60
Done  61
Done  62
Done  63
Done  64
Done  65
Done  66
Done  67
Done  68
Done  69
Done  70
Done  71
Done  72
Done  73
Done  74
Done  75
Done  76
Done  77
Done  78
Done  79
Done  80
Done  81
Done  82
Done  83
Done  84
Done  85
Done  86
Done  87
Done  88
Done  89
Done  90
Done  91
Done  92
Done  93
Done  94
Done  95
Done  96
Done  97
Done  98
Done  99
Done  100
Done  101
Done  102
Done  103
Done  104
Done  105
Done  106
Done  107
Done  108
Done  109
Don

In [19]:
unique_classes = training_data.aggregate_array('class').distinct().getInfo()

In [9]:
unique_classes

[3, 2]

In [12]:
task = ee.batch.Export.table.toAsset(
    collection=result,
    description='Classification',
    assetId="projects/ee-raman/assets/refined_all_" + suffix
)

# Start the task
task.start()

In [9]:
def add_property_count(feature):
    return feature.set('num_properties', feature.geometry().area())

fc_with_counts = rest_with_map.map(add_property_count)

# Get the maximum property count
max_prop_count = fc_with_counts.aggregate_max('num_properties')

In [10]:
max_prop_count.getInfo()

126930.1696973555

In [11]:
126930/1e6

0.12693